In [1]:
from __future__ import print_function
%matplotlib inline
import urllib.request
from pycocotools.coco import COCO
import os, sys, zipfile
import shutil
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import cv2
import csv

In [2]:
# load annotations
coco = COCO("../../annotations/instances_train2017.json");
coco_kpts = COCO("../../annotations/person_keypoints_train2017.json")

loading annotations into memory...
Done (t=40.20s)
creating index...
index created!
loading annotations into memory...
Done (t=21.49s)
creating index...
index created!


In [3]:
# get image ids of images containing person
person_IDs = coco.getCatIds(catNms = ["person"])
img_IDs = coco.getImgIds(catIds = person_IDs)

In [4]:
print(len(img_IDs))

64115


In [5]:
def get_coordinates(anns):
    coordinates = np.zeros(((int)(len(anns)/2), 2))
    for i in range(0, (int)(len(anns)/2)):
        coordinates[i][0] = anns[2*i]
        coordinates[i][1] = anns[(2*i) +1]
    return coordinates

In [ ]:
with open('keypoints.csv', mode='w',newline='') as keypoints:
    keypts_writer = csv.writer(keypoints, delimiter = ' ')
    for i in range(14042, 20000):
        #read img
        img = coco.loadImgs(img_IDs[i])[0]
        I = io.imread(img['coco_url'])
        plt.imshow(I)
        
        # the average height found was 484 -> 480
        # the average width found was 578 -> 580
        # these height, width will be used for resizing training images
        height = I.shape[0]
        width = I.shape[1]
        
        # now get the annotations for segmentation
        annIds = coco.getAnnIds(imgIds=img['id'], catIds = person_IDs, iscrowd = None)
        anns = coco.loadAnns(annIds)

        # annotations for keypoint detection
        annIds = coco_kpts.getAnnIds(imgIds=img['id'])
        anns_kpts = coco_kpts.loadAnns(annIds)
#         coco_kpts.showAnns(anns)


        # get the annotations corresponding to the segmentation having the maximum area
#         max_area = 0
#         max_area_index = 0
#         for j in range(0, len(anns)):
#             if(anns[j]['area'] > max_area):
#                 max_area = anns[j]['area']
#                 max_area_index = j
                
#         print(max_area)
#         print(max_area_index)
        if(anns[0]['area'] < 10000):
            continue
        
        keypts_writer.writerow([(int)(anns_kpts[0]['keypoints'][15]*580/width), (int)(anns_kpts[0]['keypoints'][16]*480/height), anns_kpts[0]['keypoints'][17]])
        keypts_writer.writerow([(int)(anns_kpts[0]['keypoints'][18]*580/width), (int)(anns_kpts[0]['keypoints'][19]*480/height), anns_kpts[0]['keypoints'][20]])

        # get coordinates of segment corresponding to maximum area
        cord_segm = get_coordinates(anns[0]['segmentation'][0])

        # create mask
        mask = np.zeros(I.shape[:2] , np.uint8)

        cv2.fillConvexPoly(mask,cord_segm.astype(np.int32), color = 1)
        masked_img = cv2.bitwise_and(I, I, mask = mask.astype(np.uint8))

        # save the masked image
        masked_img = cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB)
        masked_img = cv2.resize(masked_img , (580, 480))
#         masked_img = cv2.circle(masked_img,((int)(anns_kpts[0]['keypoints'][15]*580/width),(int)(anns_kpts[0]['keypoints'][16]*480/height) ),5, [255, 0, 0], -1)
#         masked_img = cv2.circle(masked_img,((int)(anns_kpts[0]['keypoints'][18]*580/width),(int)(anns_kpts[0]['keypoints'][19]*480/height) ),5, [255, 0, 0], -1)
        cv2.imwrite("D:/train_images/" + str(str(i).rjust(6, '0')) + ".jpg", masked_img)

#         plt.imshow(masked_img)